# 处理速度滑冰数据

In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [139]:
# 选择参数
traget_datas = ["Joint Angles ZXY", "Segment Angular Velocity"]
# 选择直道或者弯道
road_type = input("请选择直道或者弯道：")
print("您选择的是：", road_type)


您选择的是： 弯道


In [165]:
# 填写0，或者1
i = input("请选择0 [Joint Angles ZXY] 或1 [Segment Angular Velocity]")
target_data = traget_datas[int(i)]
print(f"您选择的是：{i}", target_data)
event_name = "蹬冰离冰"

您选择的是：1 Segment Angular Velocity


In [166]:
# 文件路径设置
excel_dir = "/Volumes/ESD-USB/哈尔滨滑冰测试/excel"
record_excel_path = "/Volumes/ESD-USB/哈尔滨滑冰测试/RecordData/20231201_短道速滑特征时刻表_期刊投稿.xlsx"

In [167]:
# 读取记录数据的excel 表格
excelfile = pd.ExcelFile(record_excel_path)
excelfile.sheet_names
record_df = pd.read_excel(record_excel_path, sheet_name="特征时刻")

In [168]:
record_df

,日期,测试序号,姓名,穿鞋,xsens号,实际xsens号,1直道开始,1直道蹬冰开始-左,1直道蹬冰结束-左,1直道蹬冰开始-右,...,2直道蹬冰开始-右,2直道蹬冰结束-右,2直道结束,2弯道开始,2弯道蹬冰开始-左,2弯道蹬冰结束-左,2弯道蹬冰开始-右,2弯道蹬冰结束-右,1弯道结束.1,备注
0,20231019,1,lininghao,不穿鞋垫,2,2,601,825,878,881,...,2550,2638,3155,3155,3225,3279,3280,3338,3773,NaN
1,20231019,1,lininghao,不穿鞋垫,3,3,183,296,350,349,...,2478,2562,3048,3048,3118,3169,3170,3226,3399,NaN
2,20231019,1,lininghao,穿鞋垫,4,4,177,375,450,448,...,2384,2428,3351,3351,3300,3364,3366,3425,3984,NaN
3,20231019,1,lininghao,穿鞋垫,5,5,429,449,511,510,...,2390,2476,3208,3208,3274,3355,3355,3420,3871,NaN
4,20231019,2,wangshuying,不穿鞋垫,1,1,310,373,439,439,...,2811,2903,3849,3935,3935,4014,4014,4094,4316,NaN
5,20231019,2,wangshuying,不穿鞋垫,2,1,192,505,571,573,...,3005,3091,3849,3393,3394,3474,3475,3548,2581,有效步态比较少，这里的2弯道和上面的 2 弯道数据一致
6,20231019,2,wangshuying,穿鞋垫,3,3,252,365,430,430,...,2419,2497,3472,3393,3394,3474,3475,3548,3887,NaN
7,20231019,2,wangshuying,穿鞋垫,4,3,986,485,540,540,...,2577,2660,1339,1339,3394,3474,3475,3548,2276,NaN
8,20231020,1,fengtianya,不穿鞋垫,1,1,253,875,941,953,...,3329,3461,4350,4350,4528,4590,4600,4656,4596,NaN
9,20231020,1,fengtianya,不穿鞋垫,2,2,77,145,220,220,...,353,420,1392,1438,1542,1599,1590,1650,2139,NaN


In [169]:
# 构建表格表头
joints = [" Hip ", " Knee ", " Ankle "]
segments = [" Upper Leg ", " Lower Leg ", " Foot "]
seg_types = ["Left", "Right"]
seg_axis = ["x", "y", "z"]
joint_orientations = ["Flexion/Extension","Abduction/Adduction", "Internal/External Rotation"]
joint_orientations_ankle = ["Dorsiflexion/Plantarflexion","Abduction/Adduction", "Internal/External Rotation"]

# 构建列表头名称
def CreateSelectColumns(seg_type,target_data, segments, joints, joint_orientations, joint_orientations_ankle, seg_axis):
    if target_data == "Joint Angles ZXY":
        select_seg_types_colmuns = [seg_type + joint + seg_orientation
                    for joint in joints
                    for seg_orientation in ((joint_orientations_ankle if joint.strip()=="Ankle" else joint_orientations))]
    else:
        select_seg_types_colmuns = [seg_type + segment + seg_ax
                    for segment in segments
                    for seg_ax in seg_axis]
    return select_seg_types_colmuns

# 获取列表名称
Left_select_columns = CreateSelectColumns("Left",target_data, segments, joints, joint_orientations, joint_orientations_ankle, seg_axis)
Right_select_columns = CreateSelectColumns("Right",target_data, segments, joints, joint_orientations, joint_orientations_ankle, seg_axis)


In [170]:
def GetSlipMaxData(slip_times, target_data_df, Left_select_columns, Right_select_columns):
    # 计算滑行过程中的最大值
    # 左侧数据的最大值
    left_max_data_list = []
    for column_name in Left_select_columns:
        slip_max = target_data_df.loc[slip_times[0]:slip_times[1], column_name].max()
        left_max_data_list.append(slip_max)
    
    # 右侧数据的最大值
    right_max_data_list = []
    for column_name in Right_select_columns:
        slip_max = target_data_df.loc[slip_times[2]:slip_times[3], column_name].max()
        right_max_data_list.append(slip_max)

    total_max_data_list = left_max_data_list + right_max_data_list

    max_data_df = pd.DataFrame([total_max_data_list], columns=Left_select_columns+Right_select_columns)
    return max_data_df

In [ ]:
def GetSlipTorsoAngleMinData(slip_times, target_data_df, "")

In [171]:


# 读取记录数据的excel表格，获取每行的相关信息

all_persons_df = pd.DataFrame()
for record_row, row in record_df.iterrows():
    date = row["日期"]
    test_number = row["测试序号"]
    name = row["姓名"]
    xsens_number = row["实际xsens号"]

    # 拼接文件名
    excel_fname = f"{int(date)}_{int(test_number)}_{name}-00{int(xsens_number)}.xlsx"

    excel_path = os.path.join(excel_dir, excel_fname)
    target_data_df = pd.read_excel(excel_path, sheet_name=target_data)
    # angle_df 
    # 读取record_df一行数据得到的结果
    attribute_info = record_df.loc[record_row, ["姓名", "穿鞋", "xsens号", "实际xsens号"]]
    # 将 series 转换为 dataframe 并重置索引
    attribute_info_df = pd.DataFrame(attribute_info).T.reset_index(drop=True)
    # 获取蹬冰开始和结束时间
    slip_times_1 = [row[f"1{road_type}蹬冰开始-左"], row[f"1{road_type}蹬冰结束-左"], 
                    row[f"1{road_type}蹬冰开始-右"], row[f"1{road_type}蹬冰结束-右"]]
    SlipMaxData_df = GetSlipMaxData(slip_times_1, target_data_df, Left_select_columns, Right_select_columns)
    result_df_1 = pd.concat([attribute_info_df, SlipMaxData_df],axis=1 )

    slip_times_2 = [row[f"2{road_type}蹬冰开始-左"], row[f"2{road_type}蹬冰结束-左"], 
                    row[f"2{road_type}蹬冰开始-右"], row[f"2{road_type}蹬冰结束-右"]]
    SlipMaxData_df = GetSlipMaxData(slip_times_2, target_data_df, Left_select_columns, Right_select_columns)
    
    result_df_2 = pd.concat([attribute_info_df, SlipMaxData_df],axis=1)

    # 拼接第一和第二道的数据
    total_result_df = pd.concat([result_df_1, result_df_2], axis=0,ignore_index=True)
    # 拼接所有人的数据
    all_persons_df = pd.concat([all_persons_df, total_result_df], axis=0,ignore_index=True)

In [172]:
# 获取当前时间
current_time = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
# 设置路径
result_excel_dir = "/Volumes/ESD-USB/哈尔滨滑冰测试/ResultData"
result_excel_fname = f"{current_time}_{road_type}_{target_data}_{event_name}_raw.xlsx"
result_excel_path = os.path.join(result_excel_dir, result_excel_fname)
# 保存到 Excel（转置后再保存）
all_persons_df.to_excel(result_excel_path, index=False)
print(f"已完成：{result_excel_fname}")

已完成：20231202_223433_弯道_Segment Angular Velocity_蹬冰离冰_raw.xlsx
